# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [2]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [3]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution

 solution = [None, False, False, True]


In [4]:

def ind_2_n(length, tup):
    out = 0
    for i in range(length):
        out = 2*out + tup[i]
            
    return out

def ind_2_col(l, a, g, b):
    L = pow(2,l)
    return ((ind_2_n(l, a))*L + ind_2_n(l, g))*2 + b 
        
# TODO: check correctness etc
def get_at(tup, a, l):
    # indexes from the left side of tup = (t0,0, t0,1, t1,0, t1,1, t2,0, t2,1)
    return tuple(tup[2*i + a[i]] for i in range(l))
 
def tuples_fixed_proj(l, proj_ind, proj_val):
#    h = randint(1,4000)
#    if (h == 5):
#        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
#        if (h == 5):
#            print 'modified x to ', x
        tuples.append(tuple(x[:]))
    return tuples    

# Assume a solution has been already found for client's variables
# Solve an artificial program for server variables just because it has
# nice support for accessing the server vars etc

def find_set_server_vars(l, solver, p0):
    
    print 'How much is server variable space limited by this solution & correctness?'
    serv_solver = MixedIntegerLinearProgram(solver='ppl')
    
    p1 = serv_solver.new_variable(integer = True, nonnegative=True)
    
    for i in range(3):                              
        for t in itertools.product(range(2), repeat = 2*l):
            serv_solver.add_constraint(p1[(i,t)] <= 1)
            
   # server's basic constraints
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l):
                for b in range(2):

                    if (solver.get_values(p0[(i,a,g,b)]) == 1):
                        all = tuples_fixed_proj(l, a, g)

                        for j in range(3):
                            if ((b == 0) and (i == j)) or ((b == 1) and (i != j)):
                                for t in all:
                                    serv_solver.add_constraint(p1[(j, t)] == 0)

    serv_solver.set_objective(sum(p1[(i,t)] for i in range(3) 
                                  for t in itertools.product(range(2), repeat = 2*l) ))
    
    res = serv_solver.solve()
    print '# of solver vars remaining = ', res


# Hack from correctness and server security (!) 
# strength = 0 is consistent with standard LP. strength = 1 attempts to better sort things out   

def add_no_id_a(solver, p0, q1, strength = 0):
    
    if strength == -1:
        return
    
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for j in range(3):
                if (j > i):
                    # this is relaxed
                    if strength == 0:
                        solver.add_constraint(sum(q1[(i, a, zeros, b)] for b in range(2)) + sum(q1[(j, a, zeros, b)] 
                                                 for b in range(2)) <= 1)    
                    elif strength == 1:
                        for b1 in range(2):
                            for b2 in range(2):
                                solver.add_constraint(p0[(i, a, zeros, b1)] + p0[(i, a, zeros, b2)] <= 1)
                                
# TODO: change the name to something more informative.
# Hack #2 from correctness: For every i,a,b with p^i_a > 0, there exists g so that (i,a,g,b) > 0.
# This requires some trial and error. Otherwise, one xi at least remains without TO-input options
# This will eventually not be part of the client constraints. Currently used to `direct' the soutions.
    
def add_a_balance(solver, p0, q1, strength = 0):
    if strength == -1:
        return
    
    if strength == 1:    
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - 1)
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      >= 1 - pow(2, l))

    # Much weaker, but no integers..
    elif strength == 0:
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(q1[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - my_eps)
                solver.add_constraint(sum(q1[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l))
                                      <= pow(2, l) - my_eps)


# Another hack from correctness. For all b, there exists at least one value of t, so 
# that every a reading this g will output b. Thus, the resulting sum of probabilities for
# that g is 1. There may be additional g's (the g results from some V assigned to server for each output
# value).                
def add_b_balance(solver, q1, strength = 0):
    if strength == -1:
        return 
    
    if strength == 0:
        for i in range(3):                              
            for b in range(2):
                # this is a relaxation - allows for splitting among several g's
                solver.add_constraint(sum(q1[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                      for g in itertools.product(range(2), repeat = l)) >= 1)                

# only captures a small subset of constraints                
def test_linear_solutions(l):
    
        b = []
        constraints = []
        
        print 'generating constraints ...'
        # needs strengthening into 3 equations
        cur_row = [0 for i in range(2*pow(2, 2*l))]
        for a in itertools.product(range(2), repeat = l):
            cur_row[ind_2_col(l, a, zeros, 0)] = 1
            cur_row[ind_2_col(l, a, zeros, 1)] = 1
        constraints.append(cur_row)
        b.append(3)
        
        
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l): 
                cur_row = [0 for i in range(2*pow(2, 2*l))]
                
                cur_row[ind_2_col(l,a,zeros,0)] = 1
                cur_row[ind_2_col(l,a,zeros,1)] = 1
                cur_row[ind_2_col(l,a,g,0)] = -1
                cur_row[ind_2_col(l,a,g,1)] = -1
                b.append(0)
                constraints.append(cur_row)

                    
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(2*pow(2, 2*l))]
            b.append(1)
            for a in itertools.product(range(2), repeat = l):
                cur_row[ind_2_col(l, a, get_at(t,a,l), 1)] = 1
            constraints.append(cur_row)
            
        
        m_constr = matrix(QQ,constraints)
        right_side = vector(QQ,b)
        print 'Created linear system of rank',m_constr.rank(),'/',pow(2, 2*l + 1)
        try:
            v = m_constr\right_side
            ker = m_constr.right_kernel()
            print 'solution = ',v
            print 'non-zeros ',sum(1 for x in v if abs(x) > 0)
            print '=================================================='
            print 'Kernel = ', ker
        except:
            print 'There is no solution!'
                
                
def find_solution(l, flag, client_bound = 1, strength = 0):
    # Generate a mixed integer program.
    # variables: 
    # 0. p0^i_{a,g,b} - 0/1 variable, 1 iff. p^i_{a,g,b} can be possibly non-zero.
    # 1. p^i_v - 0/1 variable, meaning v is (possibly) chosen towards the set. 
    # May as well not be chosen, but those assigned 0 must NOT be chosen.
    # 2. p^i_a,g,b - The probability that on input i, a is picked and b is the output.
    # 3. q^i_v - The probability that on input i, v is sent to the orcale by the server.
    # 4. q^i_a,g,b Auxiliary for privacy constraints.
    
    solver = MixedIntegerLinearProgram(solver='ppl')
    
    if (flag == 'client'):
        # Type 0
        p0 = solver.new_variable(integer = True, nonnegative=True)
        # Type 2
        q1 = solver.new_variable(integer = False, nonnegative=True)
        
        # client's probability distribution
        for i in range(3):  
            solver.add_constraint(sum(q1[(i, a, zeros, b)] for a in itertools.product(range(2), repeat = l)
                                      for b in range(2)) == 1)
 
        # client's basic constraints
        for i in range(3):                              
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        solver.add_constraint(p0[(i,a,g,b)] <= 1)
                        solver.add_constraint(p0[(i,a,g,b)] - q1[(i,a,g,b)] >= 0)
                        solver.add_constraint(p0[(i,a,g,b)] - q1[(i,a,g,b)] <= 1 - my_eps)

        # Look for a support as small as possible: to remove later.                
        solver.add_constraint(sum(p0[(i, a, g, b)] for i in range(3)
                              for a in itertools.product(range(2), repeat = l) 
                              for g in itertools.product(range(2), repeat = l)    
                              for b in range(2)) <= client_bound)               
                
        # Enforce the existence of p^i_a's                      
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):              
                    solver.add_constraint(q1[(i,a,zeros,0)] + q1[(i,a,zeros,1)] 
                                          - q1[(i,a,g,0)] -  q1[(i,a,g,1)] == 0)     
                
        # correctness constraints
        for t in itertools.product(range(2), repeat = 2*l):
            solver.add_constraint(sum(q1[(i, a, get_at(t,a,l) ,1)] 
                                      for a in itertools.product(range(2), repeat = l)
                                      for i in range(3)) == 1)
         
        
        add_no_id_a(solver, p0, q1, 1)             
        
        add_a_balance(solver, p0, q1, 1)
        
        add_b_balance(solver, q1, 0) 
        
        
        
    elif (flag == 'server'):
        
        # Type 1
        p1 = solver.new_variable(integer = True, nonnegative=True)
        # Type 3
        p2 = solver.new_variable(integer = False, nonnegative=True)

        # Type 4
        aux = solver.new_variable(integer = False, nonnegative=True)
    
    
        # server basic constraints - Make sure these are 0-1. Also make sure the probabilities of vectors 
        # not in the support of p1 are 0 
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                solver.add_constraint(p1[(i,t)] <= 1)
                solver.add_constraint(p1[(i,t)] - p2[(i,t)] >= 0)                  
            # server distribution constraint  
            solver.add_constraint(sum(p2[(i,t)] for t in itertools.product(range(2), repeat = 2*l)) == 1) 

        # privacy constraints
        for a in itertools.product(range(2), repeat = l):
            # validity of auxiliary simultion vectors
            for i in range(3):
                solver.add_constraint(sum(aux[(i,a,g,0)] for g in itertools.product(range(2), repeat = l)) 
                                    - sum(aux[(i,a,g,1)] for g in itertools.product(range(2), repeat = l)) == 0)

            solver.add_constraint(sum( aux[(i,a,g,0)] for i in range(3) 
                                      for g in itertools.product(range(2), repeat = l)) == 1)
        
        # connection to the sender's distributions
        for a in itertools.product(range(2), repeat = l):
            for i in range(3):
                for g in itertools.product(range(2), repeat = l):
                    all = tuples_fixed_proj(l, a, g)
                    solver.add_constraint(sum(p2[(i,t)] for t in all) 
                                    - sum(aux[(j, a, g, int(j == i))] for j in range(3)) == 0)
       
        # so far, yields a very trivial solution. We add simple correctness constraints (independent of any concrete client's input):
        # we know that the same t can not appear for i1 != i2. Also, may further assume wlog. that all-0 is in support of y1
        # We use p2 instead of p2 here, which would not be a linear program.
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                for j in range(3):
                    if j<i:
                        solver.add_constraint(p2[(i,t)] + p2[(j,t)] <= 1)
            
        
        # We know that the same t can not appear for i1 != i2. Also, may further assume wlog. that all-0 is in support of y1
        # This is a relaxation of what we want
        for j in range(1,3):
            for g in itertools.product(range(2), repeat = l):
                t_s = tuples_fixed_proj(l, zeros, g)
                solver.add_constraint(sum(p2[(j, t)] for t in t_s) + sum(p2[(0, t)] for t in t_s) <= 1)
        
        
        solver.set_objective(sum(-p1[(i,t)] for i in range(3)                              
                                 for t in itertools.product(range(2), repeat = 2*l))) 
    
    elif (flag == 'corr'): 
        
        print 'testing correctness'
        # honest correctness constraints - highly specific to the equality function
        # Type 0
        p0 = solver.new_variable(integer = True, nonnegative=True)
        # Type 2
        q1 = solver.new_variable(integer = False, nonnegative=True)
        # Type 1
        p1 = solver.new_variable(integer = True, nonnegative=True)

        for i in range(3):
            solver.add_constraint(sum(p1[(i,t)] for t in itertools.product(range(2), repeat = 2*l)) >= 1)
            
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                          for b in range(2)) >= 1)
            
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for j in range(3):
                        t_s = tuples_fixed_proj(l, a, g)
                        forbidden_b = 1
                        if (i == j):
                            forbidden_b = 0
                        for t in t_s:
                            solver.add_constraint(p1[(j, t)] + p0[(i, a, g, forbidden_b)] <= 1)   

        solver.add_constraint(sum(p0[(i, a, g, b)] for i in range(3)     
                             for a in itertools.product(range(2), repeat = l) 
                             for g in itertools.product(range(2), repeat = l) 
                             for b in range(2)) >= client_bound) 

         
        solver.set_objective(sum(p0[(i, a, g, b)] for i in range(3)     
                             for a in itertools.product(range(2), repeat = l) 
                             for g in itertools.product(range(2), repeat = l) 
                             for b in range(2)) + 
                             sum(p1[(i,t)] for i in range(3) for t in itertools.product(range(2), repeat = 2*l)))    
            
    res = solver.solve()
    
    if (flag == 'client'):
        find_set_server_vars(l, solver, p0)
    
    print 'result of solution process is ',res  
    if (flag == 'server' or flag == 'corr'):
        print 'p1 values  = \n'
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                if (solver.get_values(p1[(i,t)]) > 0):
                    print (i,t),': ',solver.get_values(p1[(i,t)])
    
    if (flag == 'server'):
        print 'p2 values  = \n'
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                if solver.get_values(p2[(i,t)]) > 0:
                    print (i,t),': ',solver.get_values(p2[(i,t)])

    if (flag == 'client'):            
        print 'q1 values  = \n'
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        if solver.get_values(q1[(i, a, g, b)]) > 0:
                            print (i, a, g, b),': ',solver.get_values(q1[(i, a, g, b)]),'...', solver.get_values(p0[(i, a, g, b)])
                            
        print 'THE SUM IS ',sum(solver.get_values(p0[(i,a,g,b)]) for i in range(3)
                                    for a in itertools.product(range(2), repeat = l)
                                    for g in itertools.product(range(2), repeat = l)
                                    for b in range(2)) 
                        
                            
    if (flag == 'corr'):            
        print 'q1 values  = \n'
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                for a in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        if solver.get_values(q1[(i, a, g, b)]) > 0:
                            print (i, a, g, b),': ',solver.get_values(p0[(i, a, g, b)])
        
        
    if (flag == 'server+'):                    
        print 'aux values  = \n'
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for b in range(2):
                    for g in itertools.product(range(2), repeat = l):
                        print (i, a, g, b),': ',solver.get_values(aux[(i, a, g, b)])


                        
# main     

# global variables

my_eps = QQ(1/10000)

    
for l in range(2,3):
    import time
    t0 = time.time()
    print 'finding a solution for l=',l
    zeros = tuple((0 for i in range(l)))
    
    try:
        
        find_solution(l, 'server', 400 - i)
    finally:
        print 'time =', time.time() - t0,'\n'


finding a solution for l= 2
time = 0.0255789756775 



TypeError: Unable to coerce -I + 400 to a rational